In [1]:
# ce code sert à faire des mesures de dispersion des ondes, en utilisant l'interface python permettant de communiquer avec
# le GBF DG1022, et celui permettant de communiquer avex la camera basler
#%% import libraries

import os
import sys
import time
from time import sleep
import  threading
from multiprocessing import Queue
import platform
from PyQt5.QtGui import *

from pypylon import pylon
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

import datetime
#if platform.system() =='Windows':
import pyvisa

In [6]:
#%% functions and classes for interface DG1022
debug_mode = True

def debugPrint(message):
    if debug_mode:
        print(datetime.datetime.utcnow(),' ',message)

DEVICE = '/dev/usbtmc0'

class usbtmc:
    """ Simple implementation of USBTMC device driver """
    def __init__(self,device):
        self.device = device
        try:
            self.FILE = os.open(device, os.O_RDWR)
        except Exception:
            QMessageBox.about(None,'ERROR','Unable to find requried device\n' +device)
            exit()

    def ask(self, message=""):
        """ Send a query to the unit and read the response."""
        self.write(message)
        return self.read()


    def write(self,command):
        """Send an arbitrary command directly to the device"""
        os.write(self.FILE,command)
        sleep(0.2)

    def read(self, length = 4000):
        """ Read an arbitary amount of data directly from the device"""
        return os.read(self.FILE, length)

    def getName(self):
        """ Return the unique identifier based on the *IDN? query"""
        seljf.write("*IDN?")
        sleep(0.10)
        return self.read(300)

    def sendReset(self):
        self.write("*RST")

    def close(self):
        """ Returns the ability for the user to press buttons"""
        self.write("SYST:LOC")


SINE,SQUARE,RAMP,PULSE,NOISE,ARB = range(0,6)
CH1, CH2 = range(6,8)
VPP,VRMS,DBM = range(8,11)

class RigolDG:
    """ Class to control the Rigol DS1000 series oscilloscope"""
    def __init__(self,device):
        self.meas = None
        if platform.system() =='Windows':
            rm = pyvisa.ResourceManager()
            self.meas = rm.open_resource(device)
        else:
            self.meas = usbtmc(device)
        # increase timeout
        self.meas.timeout = 5000

        self.syncVoltage=False
        self.syncFreq=False
        self.syncFreqRatio1_2=1.0
        self.syncVoltageRatio1_2=1.0
        self.reset()


    def getName(self):
        self.write("*IDN?")
        sleep(0.01)
        return self.read()

    def reset(self):
        """ Reset the instrument"""
        self.write("*RST")


    def setFunc(self, function = None, channel = None):
        """ This sets the current function of the generator, SINE, SQUARE, NOISE, RAMP"""
        if channel == None:
            channel = CH1
        if channel == CH1:
            if function == None:
                function = SINE
            msg = "FUNC "
            if(function == SINE):
                msg+="SIN"
            elif(function == SQUARE):
                msg += "SQU"
            elif(function == RAMP):
                msg += "RAMP"
            elif(function == NOISE):
                msg += "NOIS"

            if len(msg) > 6:
                self.write(msg)

        elif channel == CH2:
            if function == None:
                function = SINE
            msg = "FUNC:CH2 "
            if(function == SINE):
                msg+="SIN"
            elif(function == SQUARE):
                msg += "SQU"
            elif(function == RAMP):
                msg += "RAMP"
            elif(function == NOISE):
                msg += "NOIS"

            if len(msg) > 9:
                self.write(msg)
 

    def enableChan1(self, en = True):
        """ This function enables and checks the output of the channel. Ocassionally the screen will time out and the program would have to resend the command to ensure complience. This function only needs to be called once and will retry several times if needed"""
        msg = "OUTPut1 "
        if en:
            msg += "ON"
        else:
            msg += "OFF"
        self.write(msg)
        """attempts = 1
        maxattempts = 5
        

        if en:
            while (self.ask("OUTP?") == "OFF\n\r") and en:
                self.write(msg)
                attempts += 1
                if attempts >= maxattempts:
                    raise ValueException("Unable to Enable Channel 1!")

        else:
            while (self.ask("OUTP?") == "ON\n\r") and not en:
                self.write(msg)
                attempts +=1
                if attempts >= maxattempts:
                    raise ValueException("Unable to Disable Channel 1!")
        """

    def enableChan2(self, en=True):
        """ This function enables and checks the output of the channel. Ocassionally the screen will time out and the program would have to resend the command to ensure complience. This function only needs to be called once and will retry several times if needed"""
        msg = "OUTP:CH2 "
        if en:
            msg += "ON"
        else:
            msg += "OFF"
        self.write(msg)

        """self.Chan2_ON = en
        attempts = 1
        if en:
            while (self.ask("OUTP:CH2?") == "OFF\n\r") and en:
                self.write(msg)
                attempts += 1
                if attempts >= 5:
                    raise ValueException("Unable to Enable Channel 2!")

        else:
            while (self.ask("OUTP:CH2?") == "ON\n\r") and not en:
                self.write(msg)
                attempts +=1
                if attempts >= 5:
                    raise ValueException("Unable to Disable Channel 2!")
        """
    def isChan1Enabled(self):
        """ Check if we've got the ouput enabled"""
        ans = self.ask("OUTP?")
        if ans == 'ON\n\r':
            return True
        else:
            return False

    def isChan2Enabled(self):
        """ Check if we've got the ouput enabled"""
        ans = self.ask("OUTP:CH2?")
        if ans == 'ON\n\r':
            return True
        else:
            return False


    def setFreqHz(self, value=10000.0 ,channel =None):
        """ Set the frequency of the channel. Must provide frequency as a Hz value, ie 0.01Hz or 100000000Hz"""
        if(channel == None):
            if self.syncFreq:
                msg="FREQ "+str(value)
                self.write(msg)
                msg="FREQ:CH2 "+str(value * self.syncFreqRatio1_2)
                self.write(msg)
                return
            else:
                channel = CH1
        if channel == CH1:
            msg = "FREQ "+str(value)

            self.write(msg)
        elif channel == CH2:
            msg = "FREQ:CH2 " + str(value)
            self.write(msg)
        else:
            print("Unsupported channel selection")

    def write(self, message=""):
        """ send the message using the low level control"""
        self.meas.write(message)

    def read(self):
        """ read the message using the low level control"""
        return self.meas.read()

    def ask(self, message=""):
        """ request a response using the low level control"""
        self.write(message)
        sleep(0.1)
        return self.read()

    def setVoltageUnits(self,unit="VPP",channel = None):
        """ Provide string representation of voltage units"""
        if channel == None:
            if self.syncVoltage:
                msg="VOLT:UNIT "+unit
                self.write(msg)
                msg="VOLT:UNIT:CH2 " + unit
                self.write(msg)
                return

            else:
                channel = CH1

        if channel == CH1:
            msg = "VOLT:UNIT " + unit
            self.write(msg)
            msg = "VOLT "+ str(value)
            self.write(msg)
        elif channel == CH2:
            msg = "VOLT:UNIT:CH2 " + unit
            self.write(msg)
            msg = "VOLT:CH2 "+ str(value)
            self.write(msg)

        else:
            print("Unsupported channel selection")


    def setVoltage(self, value = 1.0, channel = None , offset=None):
        """ Set the current voltage and offset for the given channel. If no Channel specified, channel one is used by default"""
        if channel == None:
            if self.syncVoltage:
                msg="VOLT " + str(value)
                self.write(msg)

                msg="VOLT:CH2 "+str((float(value * self.syncVoltageRatio1_2)))
                sleep(0.1)
                self.write(msg)
                if offset is not None:
                    msg = "VOLT:OFFS " + offset
                    self.write(msg)
                if offset is not None:
                    msg = "VOLT:OFFS:CH2 " + str(offset)
                    self.write(msg)
                return

            else:
                channel = CH1

        if channel == CH1:
            msg = "VOLT "+ str(value)
            self.write(msg)
            if offset is not None:
                msg = "VOLT:OFFS " + offset
                self.write(msg)

        elif channel == CH2:
            msg = "VOLT:CH2 "+ str(value)
            self.write(msg)
            if offset is not None:
                msg = "VOLT:OFFS:CH2 " + str(offset)
                self.write(msg)
        else:
            print("Unsupported channel selection")

    def disconnect(self):
        """ Call this when you're done using the unit"""
        self.meas.close()


    def readVoltage(self, channel=None):
        """Read the current value, Channel one used when no channel specified"""
        if channel == None:
            channel = CH1
        volt = 0.0
        if channel == CH1:
            v1 = self.ask("VOLT?")
            end = len(v1)
            volt = float(v1[:end-1])
        elif channel == CH2:
            v2= self.ask("VOLT:CH2?")
            end = len (v2)
            volt = float(v2[4:end-1])
        return volt

    def readFreq(self, channel=None):
        """Read the current value, Channel one used when no channel specified"""
        if channel == None:
            channel = CH1
        #freq = 0.01
        if channel == CH1:
            f1 = self.ask("FREQ?")
            end = len(f1)
            freq = float(f1[:end-1])
        elif channel == CH2:
            f2 = self.ask("FREQ:CH2?")
            end = len(f2)
            freq = float(f2[4:end-1])
        return freq

    def readFunc(self,channel=None):
        """Read the current value, Channel one used when no channel specified"""
        if channel == None:
            channel = CH1
        func =""
        retValue = None
        if channel == CH1:
            func = self.ask("FUNC?")
        elif channel == CH2:
            func = self.ask("FUNC:CH2?")
        if "SIN" in func:
            retValue = SINE
        elif "SQU" in func:
            retValue = SQUARE
        elif "RAMP" in func:
            retValue = RAMP
        elif "NOIS" in func:
            retValue = NOISE
        return retValue

    def syncVoltages(self, sync=True, ratio_CH1 = 1.0, ratio_CH2=1.0):
        """ Handle the sincronisation of the channels here"""
        if sync:
            self.syncVoltage = True
            self.syncVoltageRatio1_2 = ratio_CH2 / ratio_CH1
        else:
            self.syncVoltage = False

    def syncFrequency(self,sync=True, ratio_CH1 = 1.0, ratio_CH2 = 1.0):
        """ Handle the sincronisation of the channels here"""
        if sync:
            self.syncFreq=True
            self.syncFreqRatio1_2 = ratio_CH2 / ratio_CH1
        else:
            self.syncFreq = False

def GetDG1022Device():
    """ Call this function to get cross-platform access for the function generator"""
    if platform.system() == 'Linux':
        return RigolDG('/dev/usbtmc0')
    elif platform.system() == 'Windows':
        rm = pyvisa.ResourceManager()  # This will work with both NI-VISA and pyvisa-py
        usb = list(filter(lambda x: 'USB' in x, rm.list_resources()))
        if len(usb) != 1:
            print('Unable to specify instruments. Please check connection and try again.')
            sys.exit(-1)
        return RigolDG(usb[0])




In [7]:
#%% fonction de triggering
def configure_external_trigger(camera,idx_cam,isopen=True):
    # Example for setting the TriggerSource to SoftwareSignal1
    if isopen==False:
        camera.Open()

    # Ensure the camera is not grabbing
    if camera.IsGrabbing():
        camera.StopGrabbing()


    camera.TriggerMode.SetValue('On')
    camera.TriggerSource.SetValue(list_trigger_source[idx_cam])
    camera.TriggerDelay.SetValue(list_trigger_delay[idx_cam])
    camera.TriggerSelector.SetValue("FrameStart")

    if isopen==False:
        camera.Close()


In [9]:
# Create a CameraFactory instance
tl_factory = pylon.TlFactory.GetInstance()

# Choose the devices to use
device_count = tl_factory.EnumerateDevices()
NUM_MAX_CAMERAS = len(list(device_count))

list_camera_SN = ['40300722','40437120']

#cam_array_temp = pylon.InstantCameraArray(NUM_MAX_CAMERAS)
NUM_CAMERAS = len(list_camera_SN)
cam_array = pylon.InstantCameraArray(NUM_CAMERAS)

cam_info = tl_factory.GetInstance().EnumerateDevices()
selected_cameras_indices = []
for i in range(len(list(cam_info))):
    camera = cam_info[i]
    if camera.GetSerialNumber() in list_camera_SN:
        selected_cameras_indices.append(i)

#print(selected_cameras_indices)

j=0
for i in selected_cameras_indices:
    cam_array[j].Attach(tl_factory.CreateDevice(device_count[i]))
    cam_array[j].Open()
    print('camera with index '+str(j) + ' has seral number : '+ cam_array[j].DeviceSerialNumber())
    cam_array[j].Close()
    j+=1

if len(device_count)==0:
    print('0 camera found')
    


camera with index 0 has seral number : 40300722
camera with index 1 has seral number : 40437120


In [10]:
##################################################### INPUTS & SETTINGS ######################################################

List_f_exc = np.arange(10,150,5) # par la suite les fréquences d'acquisition son choisies pour stroboscoper


list_exposure_time = [1500,1500]  # Replace with your desired exposure time in microseconds
list_width =  [1900,1900] # Replace with your desired width (px) (doit etre multiple de 16)
list_height = [400,400]   # Replace with your desired height (px) (peut etre de nimporte quelle taille)
list_offset_x = [8,8]  # Replace with your desired X offset
list_offset_y = [200,200]  # Replace with your desired Y offset
list_trigger_source = ['Line1','Line1'] 
list_trigger_delay = [0,0]  


# %% choose number of images to grab and the name of the daily folder
numberOfImagesToGrab = 100 # choisir nombre d'images
save_images = True
change_dimensions = True

##############################################################################################################################

In [11]:
#%% ensuite assigner à chaque caméra une liste pour chaque parametre à changer (les listes doivent chacune avoir NUM_CAMERAS elements)

#dailyfolder = 'D:/Vasco/Frigo_pmmh/test_interface_camera/test_two_basler/'
#dailyfolder = 'D:/Vasco/Frigo_pmmh/novembre2024/20241105/manip_f_k_membrane/'
#dailyfolder = 'D:/manip_grenoble2024/manips_relation_dispersion/20241125/'
ddd = datetime.datetime.now()
date = str(ddd.year) + str(ddd.month).zfill(2) + str(ddd.day).zfill(2)
datefolder = f'D:/Grenoble/{date}/'
dailyfolder = datefolder + 'manip_relation_dispersion/'

if os.path.exists(datefolder)==False:
    os.mkdir(datefolder)
if os.path.exists(dailyfolder)==False:
    os.mkdir(dailyfolder)


num_acquisition = 1
acquisition_folder = dailyfolder + 'Acquisition_' + str(num_acquisition) + '/'
while ((os.path.exists(acquisition_folder))|(os.path.exists(f'D:/Grenoble/{date}/manip_fracture/Acquisition_{str(num_acquisition)}/'))):
    num_acquisition += 1
    acquisition_folder = dailyfolder + 'Acquisition_' + str(num_acquisition) + '/'

print("Acquisition number : "+str(num_acquisition))

    
#if os.path.exists(savefolder+'images/')==False:
#    os.mkdir(savefolder+'images/')

# %% def the function make_acquisition (for a given frequency, for all chosen cameras)
def make_acquisition(f_exc,freq_acq):
    global cam_array
    global GBF
    try:
        cam_array.Open()

        # demonstrate some feature access
        list_str_camera_serial_number = []
        #list_freq_acq_effective = []
        for idx_cam in range(len(list(cam_array))):
            camera = cam_array[idx_cam]
            print(camera.Width.GetInc())
            print(camera.Width.GetValue())
            #camera.AcquisitionFrameRate.SetValue(freq_acq)  # Replace with your desired frame rate
            #camera.AcquisitionFrameRateEnable.SetValue(True)            
            #print('frame rate : ',camera.ResultingFrameRate.GetValue())
            camera.ExposureTime.SetValue(list_exposure_time[idx_cam])  # Replace with your desired exposure time in microseconds

            if change_dimensions==True:    
                camera.Width.SetValue(list_width[idx_cam])  # Replace with your desired width
                camera.Height.SetValue(list_height[idx_cam])  # Replace with your desired height
                camera.OffsetX.SetValue(list_offset_x[idx_cam])  # Replace with your desired X offset
                camera.OffsetY.SetValue(list_offset_y[idx_cam])  # Replace with your desired Y offset
            elif change_dimensions==False:
                new_width = camera.Width.GetValue() - camera.Width.GetInc()
                if new_width >= camera.Width.GetMin():
                    camera.Width.SetValue(new_width)
            # activer le trigger externe sur camera :
            configure_external_trigger(camera,idx_cam)
            # mettre un buffer assez grand :
            camera.MaxNumBuffer.SetValue(1000)
            str_camera_serial_number = camera.DeviceSerialNumber()
            list_str_camera_serial_number.append(str_camera_serial_number)
            #list_freq_acq_effective.append(freq_acq_effective)
            savefolder = acquisition_folder+'camera_'+str_camera_serial_number+'/' + str(f_exc)+'Hz_'+ str(np.round(freq_acq,4))+'Hz/'
            
            ## create directory to save images
            if save_images:
                if os.path.exists(acquisition_folder)==False:
                    os.mkdir(acquisition_folder)
                if os.path.exists(acquisition_folder+'camera_'+str_camera_serial_number+'/')==False:
                    os.mkdir(acquisition_folder+'camera_'+str_camera_serial_number+'/')
                if os.path.exists(savefolder)==False:
                    os.mkdir(savefolder)
                if os.path.exists(savefolder + 'images/')==False:
                    os.mkdir(savefolder + 'images/')
            ## write camera settings in a txt file    
            file = open(savefolder + 'camera_settings.txt','w')
            file.write('Serial Number : '+ list_camera_SN[idx_cam]+' \n')
            file.write('Exposure Time : ' + str(list_exposure_time[idx_cam])+' microseconds \n')
            file.write('Width : ' + str(camera.Width.GetValue())+' \n')  # Replace with your desired width
            file.write('Height : '+str(camera.Height.GetValue())+' \n')  # Replace with your desired height
            file.write('OffsetX : '+str(camera.OffsetX.GetValue())+' \n')  # Replace with your desired X offset
            file.write('OffsetY : '+str(camera.OffsetY.GetValue())+' \n')  # Replace with your desired Y offset
            file.write('Max Buffer Size : '+str(camera.MaxNumBuffer.GetValue())+' \n')
            file.close()

        # send a digital pulse to only the first camera, and all other cameras should be synchonized to it
        #send_pulse_to_camera(cam_array[0])
        # note the date and time of the beginning of the acquisition (useful in file names)
        time_acquisition = datetime.datetime.now()
        year = time_acquisition.year
        month = time_acquisition.month
        day = time_acquisition.day
        hour = time_acquisition.hour
        minutes = time_acquisition.minute
        seconds = time_acquisition.second
        str_time_acquisition = str(year)+str(month).zfill(2)+str(day).zfill(2)+'_'+str(hour).zfill(2)+str(minutes).zfill(2)+str(seconds).zfill(2)
        
        

        # start to grab images for all cameras at the same time
        #cam_array.StartGrabbing(numberOfImagesToGrab)
        array_time_elapsed = []
        time_init = time.time()
        for camera in cam_array:
            camera.StartGrabbing()
            array_time_elapsed.append(time.time() - time_init)
        array_time_elapsed = np.array(array_time_elapsed)
        print(array_time_elapsed)
        array_time_elapsed_wrtcam0 = array_time_elapsed - array_time_elapsed[0]
        array_num_images_elapsed = array_time_elapsed_wrtcam0//(1/freq_acq)
        print(array_time_elapsed_wrtcam0)
        print(array_num_images_elapsed)
            
        i = 0

        while cam_array[0].IsGrabbing(): # on a dû adapter :/ (normalement c'était juste cam_array.IsGrabbing)
            for idx_cam in range(len(list(cam_array))):
                camera = cam_array[idx_cam]
                grabResult = camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)
                if grabResult.GrabSucceeded():
                    # Access the image data.
                    print('idx_cam =',idx_cam)
                    #print("SizeX: ", grabResult.Width)
                    #print("SizeY: ", grabResult.Height)
                    img = grabResult.Array
                    #print("Gray value of first pixel: ", img[0, 0])
                    if save_images==True:
                        # Convert the NumPy array to a PIL Image object
                        image = Image.fromarray(img)
                        # Save the image as a TIFF file (black and white)
                        str_camera_serial_number = list_str_camera_serial_number[idx_cam]
                        savefolder = acquisition_folder+'camera_'+str_camera_serial_number+'/' + str(f_exc)+'Hz_' + str(np.round(freq_acq,4))+'Hz/'

                        namefile = 'Basler__'+str_camera_serial_number+'__'+str_time_acquisition+'_'+str(int(i+array_num_images_elapsed[idx_cam]-array_num_images_elapsed[-1])).zfill(4)+'.tiff' # pour les cameras qui commencent avant les autres, les premieres images auront des nombres négatifs
                        #namefile = 'Basler__'+str_camera_serial_number+'__'+str_time_acquisition+'_'+str(int(i)).zfill(4)+'.tiff'
                        if (int(i+array_num_images_elapsed[idx_cam]-array_num_images_elapsed[-1]) < 0)|(int(i+array_num_images_elapsed[idx_cam]-array_num_images_elapsed[-1])>=numberOfImagesToGrab):
                            pass
                        else:
                            image.save(savefolder+'images/'+namefile)
                        
            grabResult.Release()
            print(i)
            i+=1
            if i>=numberOfImagesToGrab + array_num_images_elapsed[-1]:
                cam_array.StopGrabbing()
                cam_array.Close()            

    finally:
        # Ensure camera is closed properly
        #cam_array.StopGrabbing()
        cam_array.Close()



Acquisition number : 1


In [12]:
List_f_exc = np.arange(10,180,5)


In [13]:
# %% "initialise la manip" : permet de tester amplitude pot vibrant avant de lancer toutes les frequences notamment

# mettre une condition sur si on veut faire de la strobo ou pas...
# fonction qui calcule la frequence d'acquisition permettant d'avoir nbpoints=100 points par periode en strobo
def calcul_freq_ech(f,npoints=100,fthr=161):
    T = 1/f
    Tvideo = T * ((npoints+1)/npoints + int(f/fthr))
    fvideo = 1/Tvideo
    return (npoints * Tvideo,fvideo)



List_freq_acq = []
for f_exc in List_f_exc:
    List_freq_acq.append(calcul_freq_ech(f_exc,npoints=100,fthr=161)[1])

#List_freq_acq = [157.1]
#List_freq_acq = np.ones(10)*57.7

# cette partie peut etre lancee avant juste pour tester la communication avec le GBF

GBF = GetDG1022Device()
sleep(1)
GBF.setVoltage(value=1,channel=CH1) # voir en fonctiion de la frequence quelle tension appliquer !!!
sleep(1)
GBF.setVoltage(value=5,channel=CH2)
sleep(1)
GBF.setFunc(function=SQUARE,channel=CH2)
sleep(1)
GBF.setFunc(function=SINE,channel=CH1)
sleep(1)
GBF.setFreqHz(value=List_f_exc[0],channel=CH1)
sleep(1)
GBF.setFreqHz(value=np.round(List_freq_acq[0],4),channel=CH2)
sleep(1)
GBF.enableChan2(en=True)
sleep(1)
GBF.enableChan1(en=True)
sleep(1)

In [14]:
#%% boucle qui change les freq
for i in range(len(List_f_exc)):
#        global camera
    # allumer channel 1 !!!!!!!!!!
    # imposer sur channel 2 la frequence freq_acq
    #if GBF.isChan2Enabled()==False:
    #    GBF.enableChan2(en=True)
    #if GBF.isChan1Enabled==False:

    #    GBF.enableChan1(en=True)
    sleep(1)
    GBF.setFreqHz(value=np.round(List_freq_acq[i],4),channel=CH2) # set freq acq ch2
    sleep(1)
    GBF.enableChan2(en=True) # activate ch2
    sleep(1)
    GBF.setFreqHz(value=List_f_exc[i],channel=CH1) # set f_exc ch1
    sleep(1)
    GBF.enableChan1(en=True) # activate ch1
    sleep(1)
    GBF.setVoltage(value = 0.7,channel=CH1) # voir en fonctiion de la frequence quelle tension appliquer !!!
    sleep(1)
    make_acquisition(List_f_exc[i],List_freq_acq[i]) # lancer acquisition pour un couple (f_exc,freq_acq) donné
    #sleep(1)
    #GBF.enableChan1(en=False) # éteindre ch1
    #sleep(1)
    #GBF.enableChan2(en=False) # éteindre ch2
    #sleep(1)


4
1920
4
1920
[0.024055   0.04616404]
[0.         0.02210903]
[0. 0.]
idx_cam = 0
idx_cam = 1
0
idx_cam = 0
idx_cam = 1
1
idx_cam = 0
idx_cam = 1
2
idx_cam = 0
idx_cam = 1
3
idx_cam = 0
idx_cam = 1
4
idx_cam = 0
idx_cam = 1
5
idx_cam = 0
idx_cam = 1
6
idx_cam = 0
idx_cam = 1
7
idx_cam = 0
idx_cam = 1
8
idx_cam = 0
idx_cam = 1
9
idx_cam = 0
idx_cam = 1
10
idx_cam = 0
idx_cam = 1
11
idx_cam = 0
idx_cam = 1
12
idx_cam = 0
idx_cam = 1
13
idx_cam = 0
idx_cam = 1
14
idx_cam = 0
idx_cam = 1
15
idx_cam = 0
idx_cam = 1
16
idx_cam = 0
idx_cam = 1
17
idx_cam = 0
idx_cam = 1
18
idx_cam = 0
idx_cam = 1
19
idx_cam = 0
idx_cam = 1
20
idx_cam = 0
idx_cam = 1
21
idx_cam = 0
idx_cam = 1
22
idx_cam = 0
idx_cam = 1
23
idx_cam = 0
idx_cam = 1
24
idx_cam = 0
idx_cam = 1
25
idx_cam = 0
idx_cam = 1
26
idx_cam = 0
idx_cam = 1
27
idx_cam = 0
idx_cam = 1
28
idx_cam = 0
idx_cam = 1
29
idx_cam = 0
idx_cam = 1
30
idx_cam = 0
idx_cam = 1
31
idx_cam = 0
idx_cam = 1
32
idx_cam = 0
idx_cam = 1
33
idx_cam = 0
idx_cam = 

idx_cam = 0
idx_cam = 1
98
idx_cam = 0
idx_cam = 1
99
4
1900
4
1900
[0.02416039 0.04978919]
[0.         0.02562881]
[0. 0.]
idx_cam = 0
idx_cam = 1
0
idx_cam = 0
idx_cam = 1
1
idx_cam = 0
idx_cam = 1
2
idx_cam = 0
idx_cam = 1
3
idx_cam = 0
idx_cam = 1
4
idx_cam = 0
idx_cam = 1
5
idx_cam = 0
idx_cam = 1
6
idx_cam = 0
idx_cam = 1
7
idx_cam = 0
idx_cam = 1
8
idx_cam = 0
idx_cam = 1
9
idx_cam = 0
idx_cam = 1
10
idx_cam = 0
idx_cam = 1
11
idx_cam = 0
idx_cam = 1
12
idx_cam = 0
idx_cam = 1
13
idx_cam = 0
idx_cam = 1
14
idx_cam = 0
idx_cam = 1
15
idx_cam = 0
idx_cam = 1
16
idx_cam = 0
idx_cam = 1
17
idx_cam = 0
idx_cam = 1
18
idx_cam = 0
idx_cam = 1
19
idx_cam = 0
idx_cam = 1
20
idx_cam = 0
idx_cam = 1
21
idx_cam = 0
idx_cam = 1
22
idx_cam = 0
idx_cam = 1
23
idx_cam = 0
idx_cam = 1
24
idx_cam = 0
idx_cam = 1
25
idx_cam = 0
idx_cam = 1
26
idx_cam = 0
idx_cam = 1
27
idx_cam = 0
idx_cam = 1
28
idx_cam = 0
idx_cam = 1
29
idx_cam = 0
idx_cam = 1
30
idx_cam = 0
idx_cam = 1
31
idx_cam = 0
idx_cam = 

idx_cam = 1
94
idx_cam = 0
idx_cam = 1
95
idx_cam = 0
idx_cam = 1
96
idx_cam = 0
idx_cam = 1
97
idx_cam = 0
idx_cam = 1
98
idx_cam = 0
idx_cam = 1
99
4
1900
4
1900
[0.02482772 0.05019164]
[0.         0.02536392]
[0. 1.]
idx_cam = 0
idx_cam = 1
0
idx_cam = 0
idx_cam = 1
1
idx_cam = 0
idx_cam = 1
2
idx_cam = 0
idx_cam = 1
3
idx_cam = 0
idx_cam = 1
4
idx_cam = 0
idx_cam = 1
5
idx_cam = 0
idx_cam = 1
6
idx_cam = 0
idx_cam = 1
7
idx_cam = 0
idx_cam = 1
8
idx_cam = 0
idx_cam = 1
9
idx_cam = 0
idx_cam = 1
10
idx_cam = 0
idx_cam = 1
11
idx_cam = 0
idx_cam = 1
12
idx_cam = 0
idx_cam = 1
13
idx_cam = 0
idx_cam = 1
14
idx_cam = 0
idx_cam = 1
15
idx_cam = 0
idx_cam = 1
16
idx_cam = 0
idx_cam = 1
17
idx_cam = 0
idx_cam = 1
18
idx_cam = 0
idx_cam = 1
19
idx_cam = 0
idx_cam = 1
20
idx_cam = 0
idx_cam = 1
21
idx_cam = 0
idx_cam = 1
22
idx_cam = 0
idx_cam = 1
23
idx_cam = 0
idx_cam = 1
24
idx_cam = 0
idx_cam = 1
25
idx_cam = 0
idx_cam = 1
26
idx_cam = 0
idx_cam = 1
27
idx_cam = 0
idx_cam = 1
28
idx_cam

93
idx_cam = 0
idx_cam = 1
94
idx_cam = 0
idx_cam = 1
95
idx_cam = 0
idx_cam = 1
96
idx_cam = 0
idx_cam = 1
97
idx_cam = 0
idx_cam = 1
98
idx_cam = 0
idx_cam = 1
99
idx_cam = 0
idx_cam = 1
100
4
1900
4
1900
[0.0199635  0.04060841]
[0.        0.0206449]
[0. 1.]
idx_cam = 0
idx_cam = 1
0
idx_cam = 0
idx_cam = 1
1
idx_cam = 0
idx_cam = 1
2
idx_cam = 0
idx_cam = 1
3
idx_cam = 0
idx_cam = 1
4
idx_cam = 0
idx_cam = 1
5
idx_cam = 0
idx_cam = 1
6
idx_cam = 0
idx_cam = 1
7
idx_cam = 0
idx_cam = 1
8
idx_cam = 0
idx_cam = 1
9
idx_cam = 0
idx_cam = 1
10
idx_cam = 0
idx_cam = 1
11
idx_cam = 0
idx_cam = 1
12
idx_cam = 0
idx_cam = 1
13
idx_cam = 0
idx_cam = 1
14
idx_cam = 0
idx_cam = 1
15
idx_cam = 0
idx_cam = 1
16
idx_cam = 0
idx_cam = 1
17
idx_cam = 0
idx_cam = 1
18
idx_cam = 0
idx_cam = 1
19
idx_cam = 0
idx_cam = 1
20
idx_cam = 0
idx_cam = 1
21
idx_cam = 0
idx_cam = 1
22
idx_cam = 0
idx_cam = 1
23
idx_cam = 0
idx_cam = 1
24
idx_cam = 0
idx_cam = 1
25
idx_cam = 0
idx_cam = 1
26
idx_cam = 0
idx_cam 

idx_cam = 0
idx_cam = 1
90
idx_cam = 0
idx_cam = 1
91
idx_cam = 0
idx_cam = 1
92
idx_cam = 0
idx_cam = 1
93
idx_cam = 0
idx_cam = 1
94
idx_cam = 0
idx_cam = 1
95
idx_cam = 0
idx_cam = 1
96
idx_cam = 0
idx_cam = 1
97
idx_cam = 0
idx_cam = 1
98
idx_cam = 0
idx_cam = 1
99
idx_cam = 0
idx_cam = 1
100
4
1900
4
1900
[0.02490401 0.05094576]
[0.         0.02604175]
[0. 1.]
idx_cam = 0
idx_cam = 1
0
idx_cam = 0
idx_cam = 1
1
idx_cam = 0
idx_cam = 1
2
idx_cam = 0
idx_cam = 1
3
idx_cam = 0
idx_cam = 1
4
idx_cam = 0
idx_cam = 1
5
idx_cam = 0
idx_cam = 1
6
idx_cam = 0
idx_cam = 1
7
idx_cam = 0
idx_cam = 1
8
idx_cam = 0
idx_cam = 1
9
idx_cam = 0
idx_cam = 1
10
idx_cam = 0
idx_cam = 1
11
idx_cam = 0
idx_cam = 1
12
idx_cam = 0
idx_cam = 1
13
idx_cam = 0
idx_cam = 1
14
idx_cam = 0
idx_cam = 1
15
idx_cam = 0
idx_cam = 1
16
idx_cam = 0
idx_cam = 1
17
idx_cam = 0
idx_cam = 1
18
idx_cam = 0
idx_cam = 1
19
idx_cam = 0
idx_cam = 1
20
idx_cam = 0
idx_cam = 1
21
idx_cam = 0
idx_cam = 1
22
idx_cam = 0
idx_cam =

...

77
idx_cam = 0
idx_cam = 1
78
idx_cam = 0
idx_cam = 1
79
idx_cam = 0
idx_cam = 1
80
idx_cam = 0
idx_cam = 1
81
idx_cam = 0
idx_cam = 1
82
idx_cam = 0
idx_cam = 1
83
idx_cam = 0
idx_cam = 1
84
idx_cam = 0
idx_cam = 1
85
idx_cam = 0
idx_cam = 1
86
idx_cam = 0
idx_cam = 1
87
idx_cam = 0
idx_cam = 1
88
idx_cam = 0
idx_cam = 1
89
idx_cam = 0
idx_cam = 1
90
idx_cam = 0
idx_cam = 1
91
idx_cam = 0
idx_cam = 1
92
idx_cam = 0
idx_cam = 1
93
idx_cam = 0
idx_cam = 1
94
idx_cam = 0
idx_cam = 1
95
idx_cam = 0
idx_cam = 1
96
idx_cam = 0
idx_cam = 1
97
idx_cam = 0
idx_cam = 1
98
idx_cam = 0
idx_cam = 1
99
idx_cam = 0
idx_cam = 1
100
idx_cam = 0
idx_cam = 1
101
4
1900
4
1900
[0.01898384 0.03940511]
[0.         0.02042127]
[0. 2.]
idx_cam = 0
idx_cam = 1
0
idx_cam = 0
idx_cam = 1
1
idx_cam = 0
idx_cam = 1
2
idx_cam = 0
idx_cam = 1
3
idx_cam = 0
idx_cam = 1
4
idx_cam = 0
idx_cam = 1
5
idx_cam = 0
idx_cam = 1
6
idx_cam = 0
idx_cam = 1
7
idx_cam = 0
idx_cam = 1
8
idx_cam = 0
idx_cam = 1
9
idx_cam = 0
idx_c

72
idx_cam = 0
idx_cam = 1
73
idx_cam = 0
idx_cam = 1
74
idx_cam = 0
idx_cam = 1
75
idx_cam = 0
idx_cam = 1
76
idx_cam = 0
idx_cam = 1
77
idx_cam = 0
idx_cam = 1
78
idx_cam = 0
idx_cam = 1
79
idx_cam = 0
idx_cam = 1
80
idx_cam = 0
idx_cam = 1
81
idx_cam = 0
idx_cam = 1
82
idx_cam = 0
idx_cam = 1
83
idx_cam = 0
idx_cam = 1
84
idx_cam = 0
idx_cam = 1
85
idx_cam = 0
idx_cam = 1
86
idx_cam = 0
idx_cam = 1
87
idx_cam = 0
idx_cam = 1
88
idx_cam = 0
idx_cam = 1
89
idx_cam = 0
idx_cam = 1
90
idx_cam = 0
idx_cam = 1
91
idx_cam = 0
idx_cam = 1
92
idx_cam = 0
idx_cam = 1
93
idx_cam = 0
idx_cam = 1
94
idx_cam = 0
idx_cam = 1
95
idx_cam = 0
idx_cam = 1
96
idx_cam = 0
idx_cam = 1
97
idx_cam = 0
idx_cam = 1
98
idx_cam = 0
idx_cam = 1
99
idx_cam = 0
idx_cam = 1
100
idx_cam = 0
idx_cam = 1
101
4
1900
4
1900
[0.02344656 0.04974771]
[0.         0.02630115]
[0. 3.]
idx_cam = 0
idx_cam = 1
0
idx_cam = 0
idx_cam = 1
1
idx_cam = 0
idx_cam = 1
2
idx_cam = 0
idx_cam = 1
3
idx_cam = 0
idx_cam = 1
4
idx_cam = 0


idx_cam = 1
64
idx_cam = 0
idx_cam = 1
65
idx_cam = 0
idx_cam = 1
66
idx_cam = 0
idx_cam = 1
67
idx_cam = 0
idx_cam = 1
68
idx_cam = 0
idx_cam = 1
69
idx_cam = 0
idx_cam = 1
70
idx_cam = 0
idx_cam = 1
71
idx_cam = 0
idx_cam = 1
72
idx_cam = 0
idx_cam = 1
73
idx_cam = 0
idx_cam = 1
74
idx_cam = 0
idx_cam = 1
75
idx_cam = 0
idx_cam = 1
76
idx_cam = 0
idx_cam = 1
77
idx_cam = 0
idx_cam = 1
78
idx_cam = 0
idx_cam = 1
79
idx_cam = 0
idx_cam = 1
80
idx_cam = 0
idx_cam = 1
81
idx_cam = 0
idx_cam = 1
82
idx_cam = 0
idx_cam = 1
83
idx_cam = 0
idx_cam = 1
84
idx_cam = 0
idx_cam = 1
85
idx_cam = 0
idx_cam = 1
86
idx_cam = 0
idx_cam = 1
87
idx_cam = 0
idx_cam = 1
88
idx_cam = 0
idx_cam = 1
89
idx_cam = 0
idx_cam = 1
90
idx_cam = 0
idx_cam = 1
91
idx_cam = 0
idx_cam = 1
92
idx_cam = 0
idx_cam = 1
93
idx_cam = 0
idx_cam = 1
94
idx_cam = 0
idx_cam = 1
95
idx_cam = 0
idx_cam = 1
96
idx_cam = 0
idx_cam = 1
97
idx_cam = 0
idx_cam = 1
98
idx_cam = 0
idx_cam = 1
99
idx_cam = 0
idx_cam = 1
100
idx_cam = 0


idx_cam = 1
54
idx_cam = 0
idx_cam = 1
55
idx_cam = 0
idx_cam = 1
56
idx_cam = 0
idx_cam = 1
57
idx_cam = 0
idx_cam = 1
58
idx_cam = 0
idx_cam = 1
59
idx_cam = 0
idx_cam = 1
60
idx_cam = 0
idx_cam = 1
61
idx_cam = 0
idx_cam = 1
62
idx_cam = 0
idx_cam = 1
63
idx_cam = 0
idx_cam = 1
64
idx_cam = 0
idx_cam = 1
65
idx_cam = 0
idx_cam = 1
66
idx_cam = 0
idx_cam = 1
67
idx_cam = 0
idx_cam = 1
68
idx_cam = 0
idx_cam = 1
69
idx_cam = 0
idx_cam = 1
70
idx_cam = 0
idx_cam = 1
71
idx_cam = 0
idx_cam = 1
72
idx_cam = 0
idx_cam = 1
73
idx_cam = 0
idx_cam = 1
74
idx_cam = 0
idx_cam = 1
75
idx_cam = 0
idx_cam = 1
76
idx_cam = 0
idx_cam = 1
77
idx_cam = 0
idx_cam = 1
78
idx_cam = 0
idx_cam = 1
79
idx_cam = 0
idx_cam = 1
80
idx_cam = 0
idx_cam = 1
81
idx_cam = 0
idx_cam = 1
82
idx_cam = 0
idx_cam = 1
83
idx_cam = 0
idx_cam = 1
84
idx_cam = 0
idx_cam = 1
85
idx_cam = 0
idx_cam = 1
86
idx_cam = 0
idx_cam = 1
87
idx_cam = 0
idx_cam = 1
88
idx_cam = 0
idx_cam = 1
89
idx_cam = 0
idx_cam = 1
90
idx_cam = 0
i

KeyboardInterrupt: 

In [32]:
    
#%% eteindre les deux channels
sleep(1)
GBF.enableChan1(en=False)
sleep(1)
GBF.enableChan2(en=False)
sleep(1)